In [ ]:
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import os
import yt
from yt.utilities.physical_constants import G, kboltz, mh
from yt.visualization.color_maps import yt_colormaps

from pygrackle import \
    FluidContainer, \
    chemistry_data

from pygrackle.one_zone import \
    MinihaloModel

from pygrackle.yt_fields import \
    prepare_grackle_data

from yt.frontends.enzo.data_structures import EnzoDataset

from yt.extensions.p2p.data_plotter import DataPlotter
from yt.extensions.p2p.model_profiles import \
    rebin_profiles, \
    find_peaks
from yt.extensions.p2p.models import \
    prepare_model

In [ ]:
plots = (
    ('density', 'time'),
    ('temperature', 'time'),
    ('pressure', 'time'),
    ('entropy', 'time'),
    ('H2_fraction', 'time'),
    ('metallicity', 'time'),
    ('mbe_ratio', 'time'),
    ('timescales', 'time'),
    ('density_growth', 'time'),
    ('H2_diss', 'time')
)

labels = {
    'time': 't [Myr]',
    'density': '$\\rho\ [g/cm^{3}]$',
    'temperature': 'T [K]',
    'entropy': 'entropy [erg cm$^{2}$]',
    'pressure': 'p [dyne/cm$^{2}$]',
    'H2_fraction': 'f$_{H2}$',
    'metallicity': 'Z [Zsun]',
    'mbe_ratio': 'M$_{gas}$ / M$_{BE}$',
    'timescales': 't [Myr]',
    'density_growth' : '$d\\rho/dt\ [g/cm^{3}/Myr]$',
    'H2_diss': 'H$_{2}$ diss [1/s]'
}

In [ ]:
pyplot.rcParams['figure.figsize'] = (15, 9)
pyplot.rcParams['font.size'] = 20

In [ ]:
yt.mylog.setLevel(40)

## Choose a star

In [ ]:
star_id = 334267081  ### first star
# star_id = 334267082  ### irradiated 1x, no heating, metal-free
# star_id = 334267083  ### irradiated 2x, heated 1x, metal-enriched (original target halo)
# star_id = 334267086  ### irradiated 2x, no heating, metal-free
# star_id = 334267090  ### irradiated 3x, heated 2x, metal-free
# star_id = 334267093  ### irradiated 4x, heated 2x, metal-free
# star_id = 334267099  ### irradiated 6x, minimal signs of heating, metal-free
# star_id = 334267102  ### irradiated 7x, heated 1x, metal-free
# star_id = 334267111  ### (target halo) irradiated 8x, multiple heatings, metal-enriched, some centering issues

In [ ]:
mass_cube_fn = f"star_cubes/star_{star_id}_mass.h5"
radius_cube_fn = f"star_cubes/star_{star_id}_radius.h5"
mass_field = 'gas_mass_enclosed'

## Load profile cube

In [ ]:
mds = yt.load(mass_cube_fn)
rds = yt.load(radius_cube_fn)

## Find peaks in Bonnor-Ebert ratio

In [ ]:
time_index = -1
field = "bonnor_ebert_ratio"
i_peaks  = find_peaks(mds, mass_field, field, time_index)

plot = False
if plot:
    bin_data = mds.data['data', mass_field][time_index]
    peak_data = mds.data['data', field][time_index]
    peak_used = peak_data > 0

    pyplot.loglog(bin_data[peak_used], peak_data[peak_used])
    pyplot.scatter(bin_data[i_peaks], peak_data[i_peaks])
    pyplot.xlabel("M$_{gas, enc}$ [Msun]")
    pyplot.ylabel("peak field")
    pyplot.show()

## Run one-zone model with Grackle

In [ ]:
grackle_pars = {
    "use_grackle": 1,
    "primordial_chemistry": 3,
    "metal_cooling": 1,
    "with_radiative_cooling": 1,
    "grackle_data_file": "cloudy_metals_2008_3D.h5",
    "h2_on_dust": 1,
    "H2_self_shielding": 0,
    "use_radiative_transfer": 1,
    "radiative_transfer_coupled_rate_solver": 0
}

#### To rerun model with different settings, run all lines from here down.

In [ ]:
prepare_grackle_data(mds, sim_type=EnzoDataset, parameters=grackle_pars)

In [ ]:
my_peak = i_peaks[0]
profile_index = my_peak
data_time = mds.data['data', 'time'].to('Myr')
first_time_index = np.where(mds.data['data', 'density'][:, my_peak] > 0)[0][0]
start_time = data_time[first_time_index]
my_fc, external_data, full_data = prepare_model(
    mds, rds, start_time, profile_index)
r_initial = full_data['data', 'radius'].to('code_length').d[0]
gas_mass = full_data['data', mass_field].to('code_mass').d[0]
# final_time = (data_time[-1] - start_time).to("s").d / my_fc.chemistry_data.time_units
final_time = 20
creation_time = mds.parameters["creation_time"]
model_creation_time = (creation_time.to("s") - start_time).d / my_fc.chemistry_data.time_units
mds.cosmology.omega_baryon = 0.0449

# Set a metallicity (comment out to use native metallicity)
# external_data['metallicity'][:] = mds.quan(1e-3, 'Zsun').to('').d

In [ ]:
# timestepping safety factor
safety_factor = 0.01

model = MinihaloModel(
    my_fc,
    external_data=external_data,
    unit_registry=mds.unit_registry,
    final_time=final_time,
    include_pressure=True,
    safety_factor=safety_factor,
    initial_radius=r_initial,
    gas_mass=gas_mass,
    include_turbulence=True,
    cosmology=mds.cosmology,
    star_creation_time=model_creation_time,
    event_trigger_fields=['RT_H2_dissociation_rate']
)
model.use_dark_matter = False
model.verbose = False
model.evolve()
mdata = model.finalize_data()

In [ ]:
### Bonnor-Ebert Mass constant
a = 1.67
b = (225 / (32 * np.sqrt(5 * np.pi))) * a**-1.5

plotter = DataPlotter(plots, labels, figsize=(15, 6), plot_func="semilogy")

my_m = full_data['data', mass_field][0]
dcolor = "black"
mcolor = "blue"
label = f"{my_m:.3f}"

data_t = mds.data['time'].to("Myr")
my_t = data_t[data_t < creation_time] - data_t[0]
model_t = mdata["time"].to("Myr")

plot_data = {
    'time': my_t,
    'entropy': full_data['data', 'entropy'].to('erg*cm**2'),
    'temperature': full_data['data', 'temperature'],
    'pressure': full_data['data', 'pressure'],
    'H2_fraction': full_data['data', 'H2_p0_fraction'],
    'H2_diss': full_data['data', 'H2_p0_dissociation_rate'],
    'metallicity': full_data['data', 'metallicity3'].to('Zsun'),
    'mbe_ratio': full_data['data', 'bonnor_ebert_ratio']
}

plotter.plot_data(plot_data, color=dcolor)
plotter.plot_datum('pressure', my_t, full_data['data', 'hydrostatic_pressure'],
                   color="green", label="hydrostatic")

plotter.plot_datum('density', my_t, full_data['data', 'density'],
                   color=dcolor, linestyle='-', label='baryon')
# plotter.plot_datum('density', my_t, full_data['data', 'dark_matter_density'],
#                    color=dcolor, linestyle='--', label='dm')
plotter.plot_datum('density', my_t, full_data['data', 'matter_density'],
                   color=dcolor, linestyle=':', label='total')

plotter.axes['timescales'].semilogy(my_t, full_data['data', 'cooling_time'].to('Myr'),
                                    color=dcolor, linestyle='-', label='t$_{cool}$')
t_ff = full_data['data', 'dynamical_time'].to('Myr')/np.sqrt(2)
plotter.plot_datum('timescales', my_t, t_ff,
                   color=dcolor, linestyle='--', label='t$_{ff}$')
plotter.plot_datum('timescales', my_t, full_data['data', 'sound_crossing_time'].to('Myr')/np.sqrt(2),
                   color=dcolor, linestyle=':', label='t$_{cs}$')

plotter.plot_datum('density_growth', my_t, full_data['data', 'density']/t_ff,
                   color=dcolor, linestyle='-', label='$\\rho/t_{ff}$')
plotter.plot_datum('density_growth', my_t, np.gradient(full_data['data', 'density'])/np.gradient(my_t),
                   color=dcolor, linestyle='--', label='$\\Delta\\rho/\\Delta t$')

plotter.plot_datum('H2_diss', my_t, full_data['data', 'H_p0_ionization_rate'],
                   color=dcolor, linestyle='--')

# t_local = my_t[1:] - my_t[0]
# x_local = np.diff(my_t) / t_ff[:-1]
# y_local = 1 / (1 - x_local/2)**2
# f_local = x_local < 2
# density_ff = density[:-1] * y_local
# growth_ff = np.gradient(density_ff) / np.diff(my_t)
# t_growth = t_local[f_local]
# ff_growth = growth_ff[f_local]

model_entropy = (mdata['temperature'] * kboltz) / (mdata['de'] / mh)**(2/3)
if 'H2I' in mdata:
    model_fH2 = mdata['H2I'] / mdata['density']
model_metallicity = (mdata['metal'] / mdata['density']).to('Zsun')
model_cs = np.sqrt(mdata['gamma'] * mdata['pressure'] / mdata['density'])
model_m_BE = (b * (model_cs**4 / G**1.5) * mdata['pressure']**-0.5).to('Msun')
model_bonnor_ebert_ratio = my_m / model_m_BE
model_time = mdata['time'].to('Myr')

model_data = {
    'time': model_time,
    'entropy': model_entropy,
    'density': mdata["density"],
    'temperature': mdata["temperature"],
    'metallicity': model_metallicity,
    'pressure': mdata["pressure"],
    'mbe_ratio': model_bonnor_ebert_ratio,
}
if 'H2I' in mdata:
    model_data['H2_fraction'] = model_fH2
if 'RT_H2_dissociation_rate' in mdata:
    model_data['H2_diss'] = mdata['RT_H2_dissociation_rate'] / my_fc.chemistry_data.time_units

plotter.plot_data(model_data, color=mcolor)

t_f = max(model_time.max(), my_t.max())
z_f = mds.cosmology.z_from_t(mds.current_time + t_f)
plotter.plot_datum('temperature', [model_time[0], t_f],
                   2.73 * (1 + np.array([model.initial_redshift, z_f])),
                   color='red')

plotter.plot_datum('timescales', model_time, -mdata['cooling_time'].to('Myr'),
                   color=mcolor, linestyle='-')
plotter.plot_datum('timescales', model_time, mdata['freefall_time'].to('Myr'),
                   color=mcolor, linestyle='--')
plotter.plot_datum('timescales', model_time, mdata['sound_crossing_time'].to('Myr'),
                   color=mcolor, linestyle=':')

model_ff_growth = (mdata["density"] / mdata["freefall_time"]).to('g/cm**3/Myr')
model_growth = (np.gradient(mdata["density"]) / np.gradient(model_time)).to('g/cm**3/Myr')
plotter.plot_datum('density_growth', model_time, model_ff_growth,
                   color=mcolor, linestyle='-')
# plotter.plot_datum('density_growth', model_time, model_growth,
#                    color=mcolor, linestyle='--')

plotter.plot_axes()
pyplot.show()
# pyplot.savefig(f'model_{star_id}.pdf', bbox_inches='tight')